In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tqdm import tqdm
import ee
import pandas as pd
from shapely.geometry import Point,Polygon

In [3]:
try:
    ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

In [4]:
CSV_PATH    = '../data/wealth_index.csv'
ALT_CSV_PATH= '../data/alt_wealth_index.csv'
L8          = 'LANDSAT/LC08/C01/T1'
L7          = 'LANDSAT/LE07/C02/T1'
L5          = 'LANDSAT/LT05/C02/T1'
L8_BANDS    = ['B2','B3','B4','B5','B6','B7']
# ALSO VALID FOR LANDSAT-5
L7_BANDS    = ['B1','B2','B3','B4','B5','B7'] 
BAND_EQUIV  = ['BLUE','GREEN','RED','NIR','SWIR1','SWIR2']

In [5]:
csv = pd.read_csv(CSV_PATH)
csv.to_csv(CSV_PATH,index=False)

In [6]:
# @Yeh, 2020 
def decode_qamask(img: ee.Image) -> ee.Image:
    '''
    Args
    - img: ee.Image, Landsat 5/7/8 image containing 'pixel_qa' band
    Returns
    - masks: ee.Image, contains 5 bands of masks
    Pixel QA Bit Flags (universal across Landsat 5/7/8)
    Bit  Attribute
    0    Fill
    1    Clear
    2    Water
    3    Cloud Shadow
    4    Snow
    5    Cloud
    '''
    qa = img.select('QA_PIXEL')
    clear = qa.bitwiseAnd(2).neq(0)  # 0 = not clear, 1 = clear
    clear = clear.updateMask(clear).rename(['pxqa_clear'])

    water = qa.bitwiseAnd(4).neq(0)  # 0 = not water, 1 = water
    water = water.updateMask(water).rename(['pxqa_water'])

    cloud_shadow = qa.bitwiseAnd(8).eq(0)  # 0 = shadow, 1 = not shadow
    cloud_shadow = cloud_shadow.updateMask(cloud_shadow).rename(['pxqa_cloudshadow'])

    snow = qa.bitwiseAnd(16).eq(0)  # 0 = snow, 1 = not snow
    snow = snow.updateMask(snow).rename(['pxqa_snow'])

    cloud = qa.bitwiseAnd(32).eq(0)  # 0 = cloud, 1 = not cloud
    cloud = cloud.updateMask(cloud).rename(['pxqa_cloud'])

    masks = ee.Image.cat([clear, water, cloud_shadow, snow, cloud])
    return masks


def mask_qaclear(img: ee.Image) -> ee.Image:
    '''
    Args
    - img: ee.Image
    Returns
    - img: ee.Image, input image with cloud-shadow, snow, cloud, and unclear
        pixels masked out
    '''
    qam = decode_qamask(img)
    cloudshadow_mask = qam.select('pxqa_cloudshadow')
    snow_mask = qam.select('pxqa_snow')
    cloud_mask = qam.select('pxqa_cloud')
    return img.updateMask(cloudshadow_mask).updateMask(snow_mask).updateMask(cloud_mask)


# SPECIFIC ERROR CORRECTOR FOR LANDSAT-7
def fill_gaps(img : ee.Image) -> ee.Image:
    return img.focal_mean(1.5,'square','pixels',2).blend(img)

In [7]:
def merge_all_bands(year):

    l8 = l7 = l5 = None
    result = None

    start_date = str(int(year)-1)+"-01-01"
    end_date = str(int(year)+1)+"-12-31"

    # LANDSAT COLLECTIONS
    if year >= 2012:
        l8 = ee.ImageCollection(L8).filterDate(start_date, end_date).filterMetadata('CLOUD_COVER', 'less_than', '10')
        result = l8.map(mask_qaclear).select(L8_BANDS, BAND_EQUIV)
    if year >= 2000:
        l7 = ee.ImageCollection(L7).filterDate(start_date, end_date)
        l7 = l7.map(mask_qaclear)
        l7 = l7.select(L7_BANDS, BAND_EQUIV)        
        if l8 is not None:
            result = l8.merge(l7)
        else:
            result = l7
    if year <= 2011:
        l5 = ee.ImageCollection(L5).filterDate(start_date, end_date)
        l5 = l5.map(mask_qaclear).select(L7_BANDS, BAND_EQUIV)
        if l7 is not None:
            result = l5.merge(l7)
        else: 
            result = l5

    # NIGHTTIME LIGHTS
    if year >= 2014:
        ntl = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate(start_date, end_date)
        ntl = ntl.select(['avg_rad'],['NTL'])
    else:
        ntl = ee.ImageCollection("NOAA/DMSP-OLS/NIGHTTIME_LIGHTS").filterDate(start_date,end_date)
        ntl = ntl.select(['stable_lights'],['NTL'])
    result = result.combine(ntl)

    # DEM 
    dem = ee.ImageCollection("COPERNICUS/DEM/GLO30").filterDate('2014-01-01', '2014-12-31')
    dem.select('DEM')
    result = result.combine(dem)
    
    return result.median()

# Exceptions in naming convention
COUNTRY_NAME_EXCEPTIONS={
            'democratic_republic_of_congo': 'Congo, Dem Rep of the',
            'cote_d_ivoire': "Cote d'Ivoire"
        }

def get_chunked_country_bounds(country, margin=0.1):
    countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
    if country in ['democratic_republic_of_congo', 'cote_d_ivoire']:
        country = COUNTRY_NAME_EXCEPTIONS[country]
    else:
        country = str(country).replace('_',' ').title()
    bounds = countries.filterMetadata('country_na','equals',country)
    bounds = bounds.geometry().bounds().getInfo()['coordinates'][0]
    xmin = bounds[0][0]
    ymin = bounds[0][1]
    xmax = bounds[2][0]
    ymax = bounds[2][1]
    xhalf = (xmax-xmin)/2
    yhalf = (ymax-ymin)/2
    final_bounds=[
        ee.Geometry.Polygon([
            [xmin-margin, ymin-margin],
            [xhalf+margin, ymin-margin],
            [xhalf+margin, yhalf+margin],
            [xhalf-margin, yhalf+margin],
            [xmin-margin, ymin-margin],
        ]),
        ee.Geometry.Polygon([
            [xmin-margin, yhalf-margin],
            [xhalf+margin, yhalf-margin],
            [xhalf+margin, ymax+margin],
            [xmin-margin, ymax+margin],
            [xmin-margin, yhalf-margin],
        ]),
        ee.Geometry.Polygon([
            [xhalf-margin, ymin-margin],
            [xmax+margin, ymin-margin],
            [xmax+margin, yhalf+margin],
            [xhalf-margin, yhalf+margin],
            [xhalf-margin, ymin-margin],
        ]),
        ee.Geometry.Polygon([
            [xhalf-margin, yhalf-margin],
            [xmax+margin, yhalf-margin],
            [xmax+margin, ymax+margin],
            [xhalf-margin, ymax+margin],
            [xhalf-margin, yhalf-margin],
        ]),
    ]
    return final_bounds

NOTE: `earth-engine api` limits the number of export tasks in queue to 3000.  
Use the printed information to start from closest checkpoint.

In [9]:
# Checkpoint values
YEAR=1996               # DEFAULT: 1996
COUNTRY='angola'      # DEFAULT: angola

csv = csv.sort_values(by='year')
for year in csv.year.unique():
    if year < YEAR:
        continue
    sub_csv = csv[ csv.year==year ]
    raw_image = merge_all_bands(int(year))
    print(year, sub_csv.country.unique())
    for country in sub_csv.country.unique():
        sub_csv = sub_csv.sort_values(by='country')
        if year==YEAR and country < COUNTRY:
            continue
        print(country)
        country_bounds = get_chunked_country_bounds(country)
        sub_csv_ = sub_csv[ sub_csv.country==country ]
        for _,row in sub_csv_.iterrows():
            for i in range(len(country_bounds)):
                image = raw_image.clip(country_bounds[i])
                name = str(country)+'_'+str(year)+'_'+str(i)
                task = ee.batch.Export.image.toDrive(**{
                    'image': image,
                    'folder': 'landsat_ntl_dem',
                    'crs' : 'EPSG:3857',
                    'fileNamePrefix' : name,
                    'fileFormat': 'GeoTIFF',
                    'scale': 30,
                    'region': country_bounds[i],
                    'maxPixels':1e13
                })
                task.start()

1996 ['mali']
mali


KeyboardInterrupt: 